# AnnotatorAgent Test Notebook

This notebook demonstrates how to test the AnnotatorAgent for mapping clinical trial conditions to MeSH terms.

## Setup and Imports

In [1]:
import os
import sys
import pandas as pd
from dotenv import load_dotenv

# Add project root to path
sys.path.append(os.path.dirname(os.getcwd()))

# Load environment variables
load_dotenv()

# Import the agent
from annotator.agent import AnnotatorAgent
from annotator.schema import TaskStatus

[2025-11-09 18:33:15] INFO     - root - ================================================================================
[2025-11-09 18:33:15] INFO     - root - Logging initialized
[2025-11-09 18:33:15] INFO     - root - Log level: INFO
[2025-11-09 18:33:15] INFO     - root - Log directory: /Users/joshziel/Documents/Coding/glp-1-landscape/logs
[2025-11-09 18:33:15] INFO     - root - Console logging: True
[2025-11-09 18:33:15] INFO     - root - ================================================================================


## Verify Environment Variables

In [2]:
# Check that required environment variables are set
print("Environment Variables:")
print(f"LOCAL_LLM_URL: {os.getenv('LOCAL_LLM_URL')}")
print(f"LOCAL_LLM: {os.getenv('LOCAL_LLM')}")
print(f"DATA_LOC: {os.getenv('DATA_LOC')}")

# Check if mesh mappings file exists
mesh_path = os.path.join(os.getenv('DATA_LOC', ''), 'mesh_term_mappings.csv')
print(f"\nMeSH mappings file exists: {os.path.exists(mesh_path)}")
if os.path.exists(mesh_path):
    print(f"Path: {mesh_path}")

Environment Variables:
LOCAL_LLM_URL: http://127.0.0.1:1234/v1
LOCAL_LLM: ibm/granite-3.2-8b
DATA_LOC: /Users/joshziel/Documents/Coding/glp-1-landscape/data

MeSH mappings file exists: True
Path: /Users/joshziel/Documents/Coding/glp-1-landscape/data/mesh_term_mappings.csv


## Load Data

In [3]:
# Load cleaned trials data
data_path = os.path.join(os.getenv('DATA_LOC', ''), 'cleaned_trials.pkl')

if os.path.exists(data_path):
    df = pd.read_pickle(data_path)
    print(f"Loaded {len(df)} trials")
    print(f"Columns: {df.columns.tolist()}")
else:
    print(f"Data file not found at: {data_path}")
    print("Creating sample data for testing...")
    # Create minimal test data
    df = pd.DataFrame({
        'nct_id': ['NCT00001', 'NCT00002', 'NCT00003'],
        'brief_title': ['Test Study 1', 'Test Study 2', 'Test Study 3'],
        'official_title': ['Official Test 1', 'Official Test 2', 'Official Test 3'],
        'brief_summary': ['Summary 1', 'Summary 2', 'Summary 3'],
        'detailed_description': [None, None, None],
        'primary_outcomes': [['Outcome 1'], ['Outcome 2'], ['Outcome 3']],
        'conditions': [['Diabetes'], ['Obesity'], []],
        'matched_conditions': [None, None, None]
    })

Loaded 1256 trials
Columns: ['nct_id', 'official_title', 'brief_title', 'acronym', 'overall_status', 'start_date', 'completion_date', 'last_update_date', 'brief_summary', 'detailed_description', 'conditions', 'interventions', 'intervention_types', 'lead_sponsor', 'sponsor_class', 'collaborators', 'countries', 'facilities', 'min_age', 'max_age', 'gender', 'healthy_volunteers', 'study_type', 'phase', 'enrollment', 'primary_outcomes', 'secondary_outcomes', 'study_url', 'citation', 'drug_name', 'cleaned_sponsor', 'cln_start_date', 'cln_completion_date', 'duration', 'matched_conditions']


In [4]:
# Use a small subset for testing
test_df = df
print(f"Testing with {len(test_df)} trials")
print("\nSample data:")
test_df[['nct_id', 'conditions', 'matched_conditions']].head()

Testing with 1256 trials

Sample data:


,nct_id,conditions,matched_conditions
958,NCT00035984,"[Diabetes Mellitus, Non-Insulin-Dependent]","[Diabetes Mellitus, Type 2 (MeSH ID:68003924)]"
819,NCT00039013,"[Diabetes Mellitus, Type 2]","[Diabetes Mellitus, Type 2 (MeSH ID:68003924)]"
1019,NCT00039026,"[Diabetes Mellitus, Non-Insulin-Dependent]","[Diabetes Mellitus, Type 2 (MeSH ID:68003924)]"
839,NCT00044668,"[Diabetes Mellitus, Type 2]","[Diabetes Mellitus, Type 2 (MeSH ID:68003924)]"
1060,NCT00044694,"[Diabetes Mellitus, Non-Insulin-Dependent]","[Diabetes Mellitus, Type 2 (MeSH ID:68003924)]"


## Create To-Do List

In [5]:
# Define tasks for the agent
tasks = [
    "Please try update matched conditions from existing mappings using your tools", 
    "Then, if necessary search for new mappings to fill in remaining unmatched conditions, using your tools"
]

print("Tasks to complete:")
for i, task in enumerate(tasks, 1):
    print(f"{i}. {task}")

Tasks to complete:
1. Please try update matched conditions from existing mappings using your tools
2. Then, if necessary search for new mappings to fill in remaining unmatched conditions, using your tools


## Initialize Agent

In [6]:
# Create the agent
agent = AnnotatorAgent(
    to_do_list=tasks,
    update=False,  # Start fresh
    data=test_df
)

print("Agent initialized successfully!")
print(f"Initial to-do list: {agent.to_do_list}")

Agent initialized successfully!
Initial to-do list: {'Please try update matched conditions from existing mappings using your tools': <TaskStatus.NOT_STARTED: 'not_started'>, 'Then, if necessary search for new mappings to fill in remaining unmatched conditions, using your tools': <TaskStatus.NOT_STARTED: 'not_started'>}


In [7]:
agent.annotator_agent_state.messages

[SystemMessage(content='You are an Annotator Agent specialized in mapping clinical trial condition data to standardized MeSH (Medical Subject Headings) terms.\n\nYour role:\n- Process clinical trial data to ensure all medical conditions are properly mapped to MeSH terms\n- Use update_condition_mapping to map unmapped conditions using existing MeSH term knowledge\n- Use search_for_condition_mapping to extract conditions from trial context when condition fields are unclear or missing\n- Work through your to-do list systematically, completing each task before moving to the next\n- Maintain accurate records of all mappings created and trials affected\n\nYour tools:\n1. update_condition_mapping: Maps unmapped conditions to existing MeSH terms using GPT-5 and validation\n2. search_for_condition_mapping: Extracts medical conditions from trial context and finds MeSH terms using mesh_mapper service\n\nYou follow a structured workflow:\n1. Check your to-do list for pending tasks\n2. Execute the 

## Run Agent - Simple Invocation

In [ ]:
# Run the workflow
print("Starting agent workflow...\n")
result = agent.workflow_graph.invoke(agent.annotator_agent_state)
print("\nWorkflow completed!")

## Run Agent - Stream Events (Alternative)

In [ ]:
# Alternative: Stream the workflow to see each step
# Uncomment to use this instead of simple invocation above

print("Starting agent workflow (streaming)...\n")
for i, event in enumerate(agent.workflow_graph.stream(agent.annotator_agent_state)):
    print(f"\n{'='*60}")
    print(f"Step {i+1}: {list(event.keys())}")
    print(f"{'='*60}")
     
     # Show details for each node
    for node_name, node_state in event.items():
        print(f"\nNode: {node_name}")
        if hasattr(node_state, 'to_do_list'):
            print(f"To-do list: {node_state.to_do_list}")
        if hasattr(node_state, 'messages'):
            last_msg = node_state.messages[-1] if node_state.messages else None
            if last_msg:
                print(f"Last message: {last_msg.content[:100]}...")  # First 100 chars
 
result = event[list(event.keys())[0]]  # Get final state
print("\nWorkflow completed!")

In [ ]:
result.keys()

## Examine Results

In [ ]:
# Check final to-do list status
print("Final To-Do List Status:")
print("-" * 50)
for task, status in result["to_do_list"].items():
    print(f"{task}: {status.value}")

In [ ]:
# View conversation history (last 10 messages)
print("\nConversation History (last 10 messages):")
print("-" * 50)
for i, msg in enumerate(result["messages"][-10:], 1):
    msg_type = type(msg).__name__
    content = msg.content if hasattr(msg, 'content') else str(msg)
    print(f"\n{i}. [{msg_type}]")
    print(f"{content}")  # First 200 chars

In [ ]:
# Compare original vs updated data
print("\nData Comparison:")
print("-" * 50)
print(f"Original data shape: {result["original_data"].shape}")
print(f"Working data shape: {result["working_data"].shape}")

print("\nMatched conditions before/after:")
comparison = pd.DataFrame({
    'nct_id': result["working_data"]['nct_id'],
    'conditions': result["working_data"]['conditions'],
    'original_matched': result["original_data"]['matched_conditions'],
    'new_matched': result["working_data"]['matched_conditions']
})
comparison

In [ ]:
# Count new mappings created
original_mapped = result['original_data']['matched_conditions'].notna().sum()
new_mapped = result['working_data']['matched_conditions'].notna().sum()

print("\nMapping Statistics:")
print("-" * 50)
print(f"Originally mapped trials: {original_mapped}")
print(f"Now mapped trials: {new_mapped}")
print(f"New mappings created: {new_mapped - original_mapped}")

## Inspect Tool Calls

In [ ]:
# Find all tool call messages
print("Tool Calls Made:")
print("-" * 50)
tool_call_count = 0

for msg in result["messages"]:
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        for tool_call in msg.tool_calls:
            tool_call_count += 1
            print(f"\n{tool_call_count}. Tool: {tool_call.get('name', 'unknown')}")
            print(f"   Args: {tool_call.get('args', {})}")

if tool_call_count == 0:
    print("No tool calls found in messages")

## Save Results (Optional)

In [ ]:
# Save the updated data
# output_path = os.path.join(os.getenv('DATA_LOC', ''), 'annotated_trials_test.pkl')
# result.working_data.to_pickle(output_path)
# print(f"Saved updated data to: {output_path}")

## Debugging: Check State at Each Step

In [ ]:
# If you need to debug, you can manually step through nodes
# Example:

# test_state = agent.annotator_agent_state
# print("Initial state:")
# print(f"To-do list: {test_state.to_do_list}")

# # Step 1: Check to-do list
# test_state = agent._check_todo_list(test_state)
# print("\nAfter check_todo_list:")
# print(f"Messages: {test_state.messages.messages[-1].content}")

# # Step 2: Work on a task
# test_state = agent._work_on_a_task(test_state)
# print("\nAfter work_on_a_task:")
# print(f"To-do list: {test_state.to_do_list}")

# # And so on...